# Evaluation all Models on Private Data

In [3]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

In [87]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [5]:
#Расчет бизнес метрики по уникальным ТТ
def antifraud_profit_metrics_by_periods_unique_tt(data, prediction, loss, target, period_name, tt_code, to_select = 30, 
                                                  n_jobs = 3):
    
    all_fraud_tt = []
    sum_losses1 = []
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_tt
        nonlocal sum_losses1
        
        data1 = data.loc[data[period_name] == period].copy()
        
        data1.sort_values(by = prediction, ascending = False, inplace = True)
        data1.reset_index(drop = True, inplace = True)
        
        data2 = data1.head(to_select)
        
        
        sum_loss1 = data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), loss].sum()
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        
        sum_losses1.append(sum_loss1)
        all_fraud_tt = all_fraud_tt + fraud_tt
        
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    from joblib import Parallel, delayed
    
    parallel = Parallel(n_jobs=n_jobs, require='sharedmem')
    
    with parallel:
        par_res = parallel((delayed(antifraud_profit_metrics_for_period_unique_tt)(period) for period in periods))
    
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1

In [6]:
#Расчет бизнес метрики без уникальных ТТ

def antifraud_profit_metrics_by_periods(data, prediction, loss, target, period_name, to_select = 30, n_jobs = 3, 
                                        only_ones = True):
    
    def antifraud_profit_metrics_for_period(data, prediction, loss, target, period_name, period, to_select = 30,
                                           only_ones = True, to_sum = True):
        
        data1 = data.loc[data[period_name] == period].copy()
        
        data1.sort_values(by = prediction, ascending = False, inplace = True)
        data1.reset_index(drop = True, inplace = True)
        
        data2 = data1.head(to_select)
        
        sum_loss = data2.loc[data2[target] == 1, loss].sum()
        sum_loss1 = data2[loss].sum()
    
        return sum_loss, sum_loss1
    
    periods = sorted(data[period_name].unique())
        
    from joblib import Parallel, delayed
    
    parallel = Parallel(n_jobs=n_jobs)
    
    with parallel:
        par_res = parallel((delayed(antifraud_profit_metrics_for_period)(data, prediction, 
                                                    loss, target, period_name, 
                                                    period, to_select, only_ones, to_sum = True) for period in periods))
    
    sums_only1 = [i[0] for i in par_res]
    sums_all = [i[1] for i in par_res]
    
    return sum(sums_only1), sum(sums_all)

In [7]:
#Расчет бизнес метрики - классический подход

def antifraud_profit_metrics_usual(data, prediction, loss, target, n_jobs = 3, only_ones = True):
    
    def for_one_threshold(data, prediction, loss, target, threshold, only_ones = True):
        
        all_len = data.shape[0]
        
        data1 = data.loc[data[prediction] >= threshold].copy()
        
        selected_len = data1.shape[0]
        
        selected_share = selected_len/all_len
        if only_ones == True:
            sum_loss = data1.loc[data1[target] == 1, loss].sum()
        else:
            sum_loss = data1[loss].sum()
        
        return selected_share, sum_loss
        
        
    from joblib import Parallel, delayed
    
    thresholds = sorted(data[prediction].unique())
    
    parallel = Parallel(n_jobs=n_jobs)
    
    with parallel:
        shares_losses = parallel((delayed(for_one_threshold)(data, prediction, loss, 
                                                              target, threshold, only_ones) for threshold in thresholds))

    shares_data = pd.DataFrame.from_records(shares_losses, columns = ['selection_rate', 'loss'])
    
    
    return shares_data

In [8]:
from scipy.stats import norm

In [9]:
def aps_ci_logit(y_true, y_scores, epsilon = 0.05):
    aps = average_precision_score(y_true, y_scores)
    n = len(y_true)
    
    nu = np.log(aps/(1-aps))
    se = ((n*aps*(1-aps))**((-1)*1/2))
    norm_se = norm.ppf(1-epsilon/2)*se
    
    exp_lower = np.exp(nu-norm_se)
    exp_upper = np.exp(nu+norm_se)
    
    lower = exp_lower/(1+exp_lower)
    upper = exp_upper/(1+exp_upper)
    
    return (lower, upper)

In [10]:
import auc_delong_xu

In [11]:
from auc_delong_xu import auc_ci_Delong

#auc, auc_var, ci = auc_ci_Delong(y_true=y_true, y_scores=y_score)

#print('ROC AUC: %s, Conf.' % auc) 
#print('Confidence Interval: %s (95%% confidence)' % str(ci))

In [12]:
def antifraud_profit_metrics_by_periods_unique_tt3(data, prediction, loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    all_fraud_dict = {}
    final_fraud_dict = {}
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[data[period_name] == period].copy()
        
        data1.sort_values(by = prediction, ascending = False, inplace = True)
        data1.reset_index(drop = True, inplace = True)
        
        data2 = data1.head(to_select)
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        
        sum_losses1.append(sum_loss1)
        
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    for period in periods:
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 #, final_fraud_dict - список ТТ

In [98]:
import os.path, time

In [99]:
from os import listdir
from os.path import isfile, join

In [100]:
dd = pd.DataFrame(columns = ['full_name', 'name', 'type', 'ctime_d', 'ctime', 'time_num'])

In [101]:
i = 0
for f in listdir('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp'): 

    if len( f.split('.')) > 1:
        tt =  f.split('.')[len( f.split('.')) - 1]
        tt0 = f.split('.' + tt)[0]
    else:
        tt = f.split('.')[0]
        tt0 = f
    ct_n = os.path.getctime('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp' + '/' + f)
    ct = datetime.strftime(datetime.fromtimestamp(ct_n), "%d.%m.%Y %H:%M:%S")
    t_n = int(datetime.fromtimestamp(ct_n).year*10000 + datetime.fromtimestamp(ct_n).month*100 + datetime.fromtimestamp(ct_n).day)
    dd = dd.append({'full_name': f, 'name': tt0, 'type':tt, 'ctime_d': ct_n, 'ctime': ct, 'time_num':t_n}, ignore_index=True)

In [102]:
dd.shape

(563, 6)

In [103]:
#d1 = dd[((dd['type'] == 'h5')) & (dd['time_num'] >= 20200908)]
d1 = dd[((dd['type'] == 'h5'))]

In [202]:
#d2 = dd[(dd['type'] == 'csv')  & (dd['time_num'] >= 20200908)][['full_name', 'name' ]]
d2 = dd[(dd['type'] == 'csv')][['full_name', 'name' ]]
d2.columns = ['csv_name', 'name' ]

In [203]:
dd_all = pd.merge(d1, d2, how='left', on='name')

In [204]:
dd_all.shape

(217, 7)

In [206]:
best_model =  ['model_spider8_internal_grid_15_3_10_100_0.0002_200512_8']

In [172]:
#лучшие по APS и PL

best_model = ['model_dense_PrivatData_c4c4_k5_t0.5_w3' ,
'model_dense_PrivateData_c3c3_k10_t0.5_w5' ,
'model_fDenseNet6_PrivateData_with_B_tests_0.0002_7_15_60_0.0002_200508_18' ,
'model_cnn_internal_grid_False_3_10_100_8_200507_7' ,
'model_cnn_internal_grid_False_5_15_60_6_200507_20' ,
'model_cnn_internal_grid_False_7_5_60_3_200501_13' ,
'model_spider6_gridTrue_0.0001_3_10_100' ,
'model_fDenseNet8_PrivateData_0.0002_7_15_100_0.0002_200513_12',   
              'model_spider8_internal_grid_20_7_5_64_0.0002_200515_2',
             
             'model_spider6_gridTrue_None_7_10_30' ,
'model_cnn_internal_grid_False_5_10_30_6_200505_13' ,
'model_spider8_internal_grid_15_5_5_60_0.0002_200508_21' , 
'model_cnn_internal_grid_False_5_15_60_8_200508_11' ,
'model_fDenseNet8_PrivateData_0.0002_7_15_60_0.0002_200511_22' ,
'model_cnn_internal_grid_False_7_15_60_3_200502_12' ,
'model_dense_PrivatData_c4c4_k5_t0.5_w3' ,
'model_dense_PrivatData_c3c3_k10_t0.5_w7' , 
'model_fDenseNet6_PrivateData_with_B_tests_0.0002_5_5_60_0.0002_200505_16'
]

In [64]:
best_model = [
    'model_spider6_gridTrue_0.0001_3_10_100' ,
    'model_spider6_grid_dropconst_0.25True_0.0001_3_10_100' ,
    'model_spider6_grid_wo_dropconv_True_0.0001_3_10_100' ,
    'model_spider8_internal_grid_20_7_5_64_0.0002_200515_2' ,
    'model_spider8_internal_grid_dropconst_0.2520_7_5_64_0.0002_200604_9' ,
    'model_spider8_internal_grid_wo_dropconv_20_7_5_64_0.0002_200602_19'
]

In [703]:
best_model = ['model_spider8_internal_wo_btest_APS20_7_5_64_0.0002_200516_17',
             'model_spider8_internal_grid_15_7_15_100_0.0002_200516_15']

In [681]:
#model_spider8_internal_wo_btest_MONEY15_5_5_60_0.0002_200516_6 
#valid_for_train:  0.8681966654708626 0.0871997826731134 | test:  0.895351112101969 0.06952349106485534
best_model = ['model_fDenseNet8_PrivateData_wo_B_tests_PL_0.0002_7_15_60_0.0002_200515_21',
             'model_spider8_internal_wo_btest_MONEY15_5_5_60_0.0002_200516_6']

In [588]:
#model_spider8_internal_grid_15_7_5_100_0.0002_200515_16 
#valid_for_train:  0.8027480001646681 0.06702631969778312 | test:  0.8283176132409 0.05003440383938146
best_model = ['model_spider8_internal_grid_15_7_5_100_0.0002_200515_16']

In [627]:
#model_cnn_best_internal_wo_btest_MONEY_False_7_15_60_3_200515_23 
#valid_for_train:  0.8995922676562436 0.06670009854090603 | test:  0.8868471519869177 0.06488889652340633

best_model = ['model_cnn_best_internal_wo_btest_MONEY_False_7_15_60_3_200515_23']

In [609]:
best_model = ['model_dense_3_3_wo_B_tests_PL']

In [536]:
best_model = ['model_fDenseNet6_PrivateData_PL_0.0002_5_5_60_0.0002_200514_23',
             'model_spider8_internal_grid_15_5_5_60_0.0002_200508_21' ,
'model_spider8_internal_grid_15_5_5_100_0.0002_200510_11' ,
'model_spider8_internal_grid_15_5_15_60_0.0002_200512_10' ,
'model_spider8_internal_grid_15_5_15_100_0.0002_200514_22' ,
'model_spider8_internal_grid_15_7_5_60_0.0002_200515_8' ,
'model_spider8_internal_grid_15_5_5_64_0.0002_200511_10' ,
'model_spider8_internal_grid_15_3_10_100_0.0002_200512_8' ,
'model_spider8_internal_grid_20_7_5_64_0.0002_200515_2'
]

In [555]:
#wo b-test
best_model = [
'model_fDenseNet6_PrivateData_wo_B_tests_PL_v2_0.0002_5_5_60_0.0002_200514_23' ,
'model_cnn_best_internal_wo_btest_MONEY_False_5_15_60_8_200515_17']

In [502]:
best_model = ['model_cnn_best_internal_wo_btest_MONEY_False_5_10_30_6_200514_15']

In [394]:
best_model = ['model_spider6_gridTrue_1e-05_5_5_64', 'model_spider6_gridTrue_0.0001_3_10_64',
'model_cnn_internal_grid_False_3_10_100_8_200507_7' ,
'model_cnn_internal_grid_False_5_15_60_6_200507_20',
'model_cnn_internal_grid_False_7_5_60_3_200501_13',
             'model_spider6_gridTrue_0.0001_3_10_100']

In [395]:
best_model = ['model_spider6_gridTrue_0.0001_3_10_100',
              'model_spider6_gridTrue_1e-05_3_10_100',
              'model_spider6_gridTrue_1e-05_3_10_64',
              'model_spider6_gridTrue_0.0001_3_10_64',
              'model_spider6_gridTrue_1e-05_3_5_30',
              'model_spider6_gridTrue_1e-05_5_10_30',
              'model_spider6_gridTrue_0.0001_5_10_100',
              'model_spider6_gridTrue_0.0001_3_10_64',

              'model_cnn_internal_grid_False_7_15_60_3_200502_12',
              'model_cnn_internal_grid_False_7_5_60_3_200501_13',
              
              'model_cnn_internal_grid_False_5_10_30_6_200505_13' ,
              'model_cnn_internal_grid_False_7_5_60_6_200501_18' ,
              'model_cnn_internal_grid_False_5_15_60_6_200507_20' ,
              'model_cnn_internal_grid_False_5_10_30_6_200505_13', 
              'model_cnn_internal_grid_False_5_15_100_6_200501_8',
              
              'model_cnn_internal_grid_False_3_10_100_8_200507_7' ,
              'model_cnn_internal_grid_False_5_15_60_8_200508_11' ,
              'model_cnn_internal_grid_False_5_10_30_8_200506_11' ,
              'model_cnn_internal_grid_False_5_5_30_8_200505_0'

              ]

In [435]:
best_model = ['model_spider6_gridTrue_0.0001_3_10_30',
              'model_spider6_gridTrue_1e-05_5_10_100',
              'model_spider6_gridTrue_1e-05_3_5_64',
              'model_spider6_gridTrue_0.0001_5_10_64',
              'model_spider6_gridTrue_1e-05_3_10_30',
              'model_spider6_gridTrue_0.0001_3_5_30',
              'model_spider6_gridTrue_1e-05_3_5_100',
              'model_spider6_gridTrue_0.0001_3_5_64',
              'model_spider6_gridTrue_None_3_5_64',
              'model_spider6_gridTrue_0.0001_5_10_30',
              'model_spider6_gridTrue_None_3_10_64',
              'model_spider6_gridTrue_1e-05_5_10_64',
              'model_spider6_gridTrue_0.0001_5_5_64',
              'model_spider6_gridTrue_None_3_10_100',
              'model_spider6_gridTrue_1e-05_7_5_64',
              'model_spider6_gridTrue_None_5_10_64',
              'model_spider6_gridTrue_None_3_10_30',
              'model_spider6_gridTrue_1e-05_7_10_30',
              'model_spider6_gridTrue_1e-05_7_10_100',
              'model_spider6_gridTrue_0.0001_5_5_30',
              'model_spider6_gridTrue_1e-05_5_5_30',
              'model_spider6_gridTrue_0.0001_3_5_100',
              'model_spider6_gridTrue_1e-05_7_5_100',
              'model_spider6_gridTrue_None_7_10_100',
              'model_spider6_gridTrue_1e-05_5_5_64',
              'model_spider6_gridTrue_0.0001_5_5_100',
              'model_spider6_gridTrue_None_3_5_30',
              'model_spider6_gridTrue_None_7_10_30',
              'model_spider6_gridTrue_None_3_5_100',
              'model_spider6_gridTrue_None_5_5_64',
              'model_spider6_gridTrue_None_7_5_100',
              'model_spider6_gridTrue_None_7_10_64',
              'model_spider6_gridTrue_0.0001_7_5_30',
              'model_spider6_gridTrue_None_5_5_100',
              'model_spider6_gridTrue_None_5_5_30',
              'model_spider6_gridTrue_1e-05_5_5_100',
              'model_spider6_gridTrue_None_5_10_30',
              'model_spider6_gridTrue_1e-05_7_5_30',
              'model_spider6_gridTrue_None_5_10_100',
              'model_spider6_gridTrue_1e-05_7_10_64',
              'model_spider6_gridTrue_None_7_5_64',
              'model_spider6_gridTrue_None_7_5_30']

In [207]:
len(set(best_model))

1

In [208]:
dd_best = dd_all[dd_all['name'].isin(best_model)]

In [209]:
dd_best.shape

(1, 7)

In [176]:
dd_best

,full_name,name,type,ctime_d,ctime,time_num,csv_name
23,model_cnn_internal_grid_False_3_10_100_8_20050...,model_cnn_internal_grid_False_3_10_100_8_200507_7,h5,1.620363e+09,07.05.2021 07:52:15,20210507,model_cnn_internal_grid_False_3_10_100_8_20050...
24,model_cnn_internal_grid_False_5_10_30_6_200505...,model_cnn_internal_grid_False_5_10_30_6_200505_13,h5,1.620212e+09,05.05.2021 13:51:10,20210505,model_cnn_internal_grid_False_5_10_30_6_200505...
30,model_cnn_internal_grid_False_5_15_60_6_200507...,model_cnn_internal_grid_False_5_15_60_6_200507_20,h5,1.620407e+09,07.05.2021 20:04:52,20210507,model_cnn_internal_grid_False_5_15_60_6_200507...
31,model_cnn_internal_grid_False_5_15_60_8_200508...,model_cnn_internal_grid_False_5_15_60_8_200508_11,h5,1.620463e+09,08.05.2021 11:36:06,20210508,model_cnn_internal_grid_False_5_15_60_8_200508...
38,model_cnn_internal_grid_False_7_15_60_3_200502...,model_cnn_internal_grid_False_7_15_60_3_200502_12,h5,1.619949e+09,02.05.2021 12:44:32,20210502,model_cnn_internal_grid_False_7_15_60_3_200502...
42,model_cnn_internal_grid_False_7_5_60_3_200501_...,model_cnn_internal_grid_False_7_5_60_3_200501_13,h5,1.619864e+09,01.05.2021 13:12:41,20210501,model_cnn_internal_grid_False_7_5_60_3_200501_...
50,model_dense_PrivatData_c3c3_k10_t0.5_w7.h5,model_dense_PrivatData_c3c3_k10_t0.5_w7,h5,1.620824e+09,12.05.2021 15:58:19,20210512,model_dense_PrivatData_c3c3_k10_t0.5_w7.csv
63,model_dense_PrivatData_c4c4_k5_t0.5_w3.h5,model_dense_PrivatData_c4c4_k5_t0.5_w3,h5,1.620736e+09,11.05.2021 15:18:58,20210511,model_dense_PrivatData_c4c4_k5_t0.5_w3.csv
67,model_dense_PrivateData_c3c3_k10_t0.5_w5.h5,model_dense_PrivateData_c3c3_k10_t0.5_w5,h5,1.620736e+09,11.05.2021 15:18:58,20210511,model_dense_PrivateData_c3c3_k10_t0.5_w5.csv
73,model_fDenseNet6_PrivateData_with_B_tests_0.00...,model_fDenseNet6_PrivateData_with_B_tests_0.00...,h5,1.620824e+09,12.05.2021 15:58:19,20210512,model_fDenseNet6_PrivateData_with_B_tests_0.00...


In [469]:
dd = pd.DataFrame(columns = ['full_name', 'name', 'type', 'ctime_d', 'ctime', 'time_num'])

In [470]:
i = 0
for f in listdir('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/best_dense/DenseNet-8-6 PL'): 

    if len( f.split('.')) > 1:
        tt =  f.split('.')[len( f.split('.')) - 1]
        tt0 = f.split('.' + tt)[0]
    else:
        tt = f.split('.')[0]
        tt0 = f
    ct_n = os.path.getctime('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/best_dense/DenseNet-8-6 PL' + '/' + f)
    ct = datetime.strftime(datetime.fromtimestamp(ct_n), "%d.%m.%Y %H:%M:%S")
    t_n = int(datetime.fromtimestamp(ct_n).year*10000 + datetime.fromtimestamp(ct_n).month*100 + datetime.fromtimestamp(ct_n).day)
    dd = dd.append({'full_name': f, 'name': tt0, 'type':tt, 'ctime_d': ct_n, 'ctime': ct, 'time_num':t_n}, ignore_index=True)

In [471]:
d1 = dd[((dd['type'] == 'h5'))]
d2 = dd[(dd['type'] == 'csv')][['full_name', 'name' ]]
d2.columns = ['csv_name', 'name' ]
dd_all = pd.merge(d1, d2, how='left', on='name')

In [472]:
dd_all

,full_name,name,type,ctime_d,ctime,time_num,csv_name
0,model_dense_4_4_wo_B_tests.h5,model_dense_4_4_wo_B_tests,h5,1.620979e+09,14.05.2021 10:58:02,20210514,model_dense_4_4_wo_B_tests.csv
1,model_fDenseNet6_PrivateData_wo_B_tests_PL_0.0...,model_fDenseNet6_PrivateData_wo_B_tests_PL_0.0...,h5,1.620979e+09,14.05.2021 10:58:02,20210514,model_fDenseNet6_PrivateData_wo_B_tests_PL_0.0...
2,model_spider6_gridTrue_None_7_10_30.h5,model_spider6_gridTrue_None_7_10_30,h5,1.620979e+09,14.05.2021 10:58:02,20210514,model_spider6_gridTrue_None_7_10_30.csv
3,RIGHT_model_spider6_internal_wo_btest_7_10_30_...,RIGHT_model_spider6_internal_wo_btest_7_10_30_...,h5,1.620979e+09,14.05.2021 10:58:02,20210514,RIGHT_model_spider6_internal_wo_btest_7_10_30_...


In [473]:
#dd_best = pd.concat([dd_all, dd_best], axis = 0)
dd_best = dd_all

In [413]:
dd_all.shape

(29, 7)

In [27]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [28]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/
D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/


In [29]:
target = 'BAD_FLAG'
index_month = 'MONTH_YEAR'
list_of_vars_for_strat = ['MONTH_YEAR']
sort_by_var = 'APPPOSID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0', 'PERIOD_7', 'LOSS_90P'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

# ***

In [30]:
new_features_data_no_vif = pd.read_csv(PATH + '/SandBox/selected_features_forest.csv', sep = '\t', index_col=0)
new_features_data_no_vif_no_sys = pd.read_csv(PATH + '/SandBox/selected_features_forest_vif_no_sys.csv', sep = '\t', index_col = 0)
categorials_data_fin = pd.read_csv(PATH + 'SandBox/categories_fin.csv', sep = '\t', index_col=0)
definition_2 = pd.read_csv(PATH + 'SandBox/definition_2.csv', index_col=0)

In [31]:
selected_features = new_features_data_no_vif
cols = selected_features['genuine variable'].drop_duplicates().to_list()

In [32]:
train_data = 'antifraud_data_train_with_2014.csv'
test_data  = 'antifraud_data_test_with_2014.csv'
val_data   = 'antifraud_data_valid_with_2014.csv'
oot_data = 'data_2013.csv'

In [33]:
raw_columns = (pd.read_csv(PATH_DATA +  train_data, sep = '&', encoding = 'utf-8', nrows = 0)).columns

In [34]:
raw_columns

Index(['Unnamed: 0', 'APPPOSID', 'PERIOD_7', 'SUB_CHANNEL', 'COUNT_APP_7',
       'COUNT_CONT_7', 'COUNT_APP_14', 'COUNT_CONT_14', 'COUNT_APP_30',
       'COUNT_CONT_30',
       ...
       'RULE_DIFF_INC_3_30', 'RULE_DIFF_INC_7_30', 'RULE_DIFF_INC_9_30',
       'COUNT_INC_60', 'RULE_DIFF_INC_3_60', 'RULE_DIFF_INC_7_60',
       'RULE_DIFF_INC_9_60', 'LOSS_90P', 'MONTH_YEAR', 'BAD_FLAG'],
      dtype='object', length=512)

In [35]:
cols_pl = [i.upper() for i in [ 'period_7', 'loss_90p']]
cols_pl

['PERIOD_7', 'LOSS_90P']

In [36]:
raw_columns_2 = raw_columns = (pd.read_csv(PATH_DATA +  oot_data, sep = '&', encoding = 'cp1251', nrows = 0)).columns

oot0   = pd.read_csv(PATH_DATA +   oot_data,   sep = '&', encoding = 'cp1251',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns_2 if c.upper() in cols])

In [37]:
cols = cols + cols_pl

In [38]:
raw_columns = (pd.read_csv(PATH_DATA +  train_data, sep = '&', encoding = 'utf-8', nrows = 0)).columns # исходные имена колонок
#dtype={c:type_cols.get(c.upper(),'float32') for c in raw_columns}                                                  # справочник для исходных колонок
train0 = pd.read_csv(PATH_DATA  +  train_data, sep = '&', encoding = 'utf-8',  #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])
test0  = pd.read_csv(PATH_DATA +  test_data,  sep = '&', encoding = 'utf-8',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])
val0   = pd.read_csv(PATH_DATA +   val_data,   sep = '&', encoding = 'cp1251',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])

raw_columns_2 = raw_columns = (pd.read_csv(PATH_DATA +  oot_data, sep = '&', encoding = 'cp1251', nrows = 0)).columns

oot0   = pd.read_csv(PATH_DATA +   oot_data,   sep = '&', encoding = 'cp1251',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns_2 if c.upper() in cols])

train0.columns = [col.upper() for col in train0.columns]
test0.columns  = [col.upper() for col in test0.columns]
val0.columns   = [col.upper() for col in val0.columns]
oot0.columns   = [col.upper() for col in oot0.columns]

In [228]:
test_check  = pd.read_csv(PATH_DATA +  test_data,  sep = '&', encoding = 'utf-8',  
                    #dtype=dtype, 
                    usecols=['PERIOD_7', 'MONTH_YEAR'])

In [229]:
test_check

,PERIOD_7,MONTH_YEAR
0,52,201903
1,70,201810
2,71,201810
3,72,201810
4,73,201810
...,...,...
185395,277,201411
185396,278,201411
185397,279,201410
185398,280,201410


In [225]:
raw_columns

Index(['APPPOSID', 'period_7', 'SUB_CHANNEL', 'count_app_7', 'count_cont_7',
       'count_app_14', 'count_cont_14', 'count_app_30', 'count_cont_30',
       'count_app_60',
       ...
       'b_test_fifth_number_6_14d', 'b_test_fifth_number_6_30d',
       'b_test_fifth_number_6_60d', 'w_test_fifth_number_6_7d',
       'w_test_fifth_number_6_14d', 'w_test_fifth_number_6_30d',
       'w_test_fifth_number_6_60d', 'loss_90p', 'bad_flag', 'month_year'],
      dtype='object', length=535)

In [39]:
col_pl = [i.upper() for i in ['APPPOSID', 'period_7', 'loss_90p', 'bad_flag' ]]
train_pl = train0[col_pl]
test_pl = test0[col_pl]
val_pl = val0[col_pl]

In [40]:
oot_pl = oot0[col_pl]

In [41]:
selected_features

,new variable,genuine variable,rule,values
0,COUNT_CONT_30,COUNT_CONT_30,Missing,NaN
1,W_TEST_AGE_6_60D,W_TEST_AGE_6_60D,Missing,NaN
2,W_TEST_AGE_ALL5_7D,W_TEST_AGE_ALL5_7D,Missing,NaN
3,B_TEST_PASP_D_3_7D,B_TEST_PASP_D_3_7D,Missing,NaN
4,B_TEST_PASP_D_3_14D,B_TEST_PASP_D_3_14D,Missing,NaN
...,...,...,...,...
161,RULE_1_TRE3_SHARE_14_bin,RULE_1_TRE3_SHARE_14,_bin,NaN
162,COUNT_CONT_FIO7_7_bin,COUNT_CONT_FIO7_7,_bin,NaN
163,BAD_FLAG,BAD_FLAG,Missing,NaN
164,MONTH_YEAR,MONTH_YEAR,Missing,NaN


In [42]:
cols_pl

['PERIOD_7', 'LOSS_90P']

In [43]:
selected_features.at[166, 'new variable' ] = 'PERIOD_7'
selected_features.at[166, 'genuine variable' ] = 'PERIOD_7'
selected_features.at[166, 'rule' ] = 'Missing'
selected_features.at[166, 'values' ] = np.nan

In [44]:
selected_features.at[167, 'new variable' ] = 'LOSS_90P'
selected_features.at[167, 'genuine variable' ] = 'LOSS_90P'
selected_features.at[167, 'rule' ] = 'Missing'
selected_features.at[167, 'values' ] = np.nan

In [45]:
selected_features

,new variable,genuine variable,rule,values
0,COUNT_CONT_30,COUNT_CONT_30,Missing,NaN
1,W_TEST_AGE_6_60D,W_TEST_AGE_6_60D,Missing,NaN
2,W_TEST_AGE_ALL5_7D,W_TEST_AGE_ALL5_7D,Missing,NaN
3,B_TEST_PASP_D_3_7D,B_TEST_PASP_D_3_7D,Missing,NaN
4,B_TEST_PASP_D_3_14D,B_TEST_PASP_D_3_14D,Missing,NaN
...,...,...,...,...
163,BAD_FLAG,BAD_FLAG,Missing,NaN
164,MONTH_YEAR,MONTH_YEAR,Missing,NaN
165,APPPOSID,APPPOSID,Missing,NaN
166,PERIOD_7,PERIOD_7,Missing,NaN


In [46]:
train = turn_variables_with_values(train0, selected_features)
test  = turn_variables_with_values(test0,  selected_features)
val   = turn_variables_with_values(val0,   selected_features)
oot = turn_variables_with_values(oot0,   selected_features)

In [47]:
y_train = train[target]
y_test = test[target]
y_val = val[target]
y_oot = oot[target]

train.drop(target, axis = 1, inplace = True)
test.drop(target, axis = 1, inplace = True)
val.drop(target, axis = 1, inplace = True)
oot.drop(target, axis = 1, inplace = True)

In [48]:
y_train = y_train.astype('int')
y_test  = y_test.astype('int')
y_val   = y_val.astype('int')
y_oot   = y_oot.astype('int')

In [49]:
necessary_fields = necessary_fields + cols_pl
necessary_fields

['BAD_FLAG', 'MONTH_YEAR', 'APPPOSID', 'PERIOD_7', 'LOSS_90P']

In [50]:
list_of_columns = selected_features['new variable'].to_list()
list_of_columns_no_sys = [c for c in list_of_columns if c not in necessary_fields]

In [51]:
attribute_list_model = attributes_list_new(train, list_of_columns_no_sys, [0.01, 0.5, 1, 2, 5, 95, 97, 
                                                                           98, 99, 99.5, 99.99])

[1.000e-04 5.000e-03 1.000e-02 2.000e-02 5.000e-02 9.500e-01 9.700e-01
 9.800e-01 9.900e-01 9.950e-01 9.999e-01]
Number of finished repetitions: 0 | time:  01.09.2021 11:34:07
Number of finished repetitions: 100 | time:  01.09.2021 11:34:54


In [52]:
list_of_categories_sel = attribute_list_model.loc[(attribute_list_model['count_dist'] == 2), 'attribute'].to_list()

In [53]:
for i in list_of_columns_no_sys:
    if i in categorials_data_fin:
        list_of_categories_sel.append(i)

In [54]:
attribute_list_model['right_border'] = attribute_list_model['99%']
attribute_list_model['left_border'] = attribute_list_model['1%']

In [55]:
st_of_columns_no_sys = [c for c in list_of_columns if c not in necessary_fields]
cols_outliers = list_of_columns_no_sys.copy()

In [56]:
X_1_2,  preproc_ = data_preprocessing_train(train, y_train, #test, y_test, 
                                                    technical_values = necessary_fields, 
                                                       categorial_list = list_of_categories_sel, drop_technical = False,
                                                       yeo_johnson = False, attribute_list = attribute_list_model, 
                                                       var_col = 'attribute',
                                                       scale = 'mean', 
                                                       median = 'val_mediana', 
                                                       
                                                       high_outlier = 'right_border', 
                                                       low_outlier = 'left_border', 
                                                    check_percentile = 5,
                                                       cols_outlier = cols_outliers, 
                                                       cut_non_out_9999 = True)

In [57]:
X_2_2 = data_preprocessing_test(test, y_test, necessary_fields, categorial_list = list_of_categories_sel, 
                                drop_technical = False, #True,
                                attribute_list = attribute_list_model, var_col = 'attribute',
                                median = 'val_mediana',
                                high_outlier = 'right_border',
                                low_outlier = 'left_border', 
                                scale =  preproc_, 
                                #yeo_johnson = None,
                                cols_outlier = cols_outliers)

In [58]:
X_3_2 = data_preprocessing_test(val, y_val, necessary_fields, categorial_list = list_of_categories_sel, 
                                drop_technical = False, #True,
                                attribute_list = attribute_list_model, var_col = 'attribute',
                                median = 'val_mediana',
                                high_outlier = 'right_border',
                                low_outlier = 'left_border', 
                                scale =  preproc_, 
                                #yeo_johnson = None,
                                cols_outlier = cols_outliers)

In [59]:
X_4_2 = data_preprocessing_test(oot, y_oot, necessary_fields, categorial_list = list_of_categories_sel, 
                                drop_technical = False, #True,
                                attribute_list = attribute_list_model, var_col = 'attribute',
                                median = 'val_mediana',
                                high_outlier = 'right_border',
                                low_outlier = 'left_border', 
                                scale =  preproc_, 
                                #yeo_johnson = None,
                                cols_outlier = cols_outliers)

In [60]:
print('train', X_1_2.shape, train.shape, len(y_train))
print('val', X_2_2.shape, test.shape, len(y_test))
print('test', X_3_2.shape, val.shape, len(y_val))
print('oot', X_4_2.shape, oot.shape, len(y_oot))

train (1507599, 167) (1507599, 167) 1507599
val (185400, 167) (185400, 167) 185400
test (187500, 167) (187500, 167) 187500
oot (84403, 167) (84403, 167) 84403


In [61]:
X_3_2.head()

,COUNT_CONT_30,W_TEST_AGE_6_60D,W_TEST_AGE_ALL5_7D,B_TEST_PASP_D_3_7D,B_TEST_PASP_D_3_14D,B_TEST_PASP_D_3_30D,B_TEST_PASP_D_3_60D,B_TEST_PASP_D_10_7D,B_TEST_PASP_D_10_30D,B_TEST_PASP_D_ALL5_30D,...,AVG_FUND_TT_30_bin,STD_SUM_TT_30_bin,STD_SUM_TT_60_bin,RULE_1_TRE3_SHARE_7_bin,RULE_1_TRE3_SHARE_14_bin,COUNT_CONT_FIO7_7_bin,MONTH_YEAR,APPPOSID,PERIOD_7,LOSS_90P
0,-0.342265,-0.244914,-0.289367,-0.327212,-0.213062,-0.702378,-0.562836,-0.645983,-0.046856,-0.000692,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,23.0,-2184.848151
1,-0.342265,-0.393576,-0.351571,-1.066225,-0.390126,-1.097082,-0.685552,-0.474433,0.269880,-0.508555,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,24.0,-2405.939969
2,-0.411660,-0.213598,0.254707,0.813310,-0.375775,0.714397,-0.909641,0.803956,1.013864,-0.519042,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,26.0,-2260.590797
3,-0.619846,-0.047949,-0.075780,-0.070572,0.008132,-0.838069,-0.087148,0.149652,-0.881406,-0.748627,...,1.0,0.0,1.0,0.0,0.0,0.0,201908.0,8299.0,27.0,-4052.320664
4,-0.481055,-0.098913,0.131363,-0.283582,-0.139454,0.018673,-0.669151,-0.071867,-0.675116,-0.210213,...,1.0,1.0,1.0,0.0,0.0,0.0,201908.0,8299.0,28.0,-4052.320664


In [57]:
print('train', X_1_2.shape, train.shape, len(y_train))
print('val', X_2_2.shape, test.shape, len(y_test))
print('test', X_3_2.shape, val.shape, len(y_val))
print('oot', X_4_2.shape, oot.shape, len(y_oot))

train (1507599, 167) (1507599, 167) 1507599
val (185400, 167) (185400, 167) 185400
test (187500, 167) (187500, 167) 187500
oot (84403, 167) (84403, 167) 84403


In [58]:
X_3_2.head()

,COUNT_CONT_30,W_TEST_AGE_6_60D,W_TEST_AGE_ALL5_7D,B_TEST_PASP_D_3_7D,B_TEST_PASP_D_3_14D,B_TEST_PASP_D_3_30D,B_TEST_PASP_D_3_60D,B_TEST_PASP_D_10_7D,B_TEST_PASP_D_10_30D,B_TEST_PASP_D_ALL5_30D,...,AVG_FUND_TT_30_bin,STD_SUM_TT_30_bin,STD_SUM_TT_60_bin,RULE_1_TRE3_SHARE_7_bin,RULE_1_TRE3_SHARE_14_bin,COUNT_CONT_FIO7_7_bin,MONTH_YEAR,APPPOSID,PERIOD_7,LOSS_90P
0,-0.342265,-0.244914,-0.289367,-0.327212,-0.213062,-0.702378,-0.562836,-0.645983,-0.046856,-0.000692,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,23.0,-2184.848151
1,-0.342265,-0.393576,-0.351571,-1.066225,-0.390126,-1.097082,-0.685552,-0.474433,0.269880,-0.508555,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,24.0,-2405.939969
2,-0.411660,-0.213598,0.254707,0.813310,-0.375775,0.714397,-0.909641,0.803956,1.013864,-0.519042,...,1.0,1.0,1.0,1.0,1.0,0.0,201909.0,8299.0,26.0,-2260.590797
3,-0.619846,-0.047949,-0.075780,-0.070572,0.008132,-0.838069,-0.087148,0.149652,-0.881406,-0.748627,...,1.0,0.0,1.0,0.0,0.0,0.0,201908.0,8299.0,27.0,-4052.320664
4,-0.481055,-0.098913,0.131363,-0.283582,-0.139454,0.018673,-0.669151,-0.071867,-0.675116,-0.210213,...,1.0,1.0,1.0,0.0,0.0,0.0,201908.0,8299.0,28.0,-4052.320664


In [62]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
#загружаем данные для 163 переменных

list_train_for = pd.read_csv(PATH + 'train_163_prep.csv', nrows = 10)
list_train_for.drop('Unnamed: 0', axis = 1, inplace = True)

col_train = list_train_for.columns.to_list()
col_train.remove(target)

In [63]:
len(col_train)

163

# ***

In [ ]:
#PATH + 'train_51_prep.csv'
#PATH + 'train_163_prep.csv'

In [17]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [18]:
y_train_163 = train_for[target]
y_test_163 = valid_for[target]
y_val_163 = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2_163 = train_for[col]
X_2_2_163 = valid_for[col]
X_3_2_163 = test_for[col]

In [208]:
X_2_2_163.shape, X_2_2.shape

((185400, 163), (185400, 163))

In [94]:
X_1_2_163.shape

(1507599, 163)

In [201]:
train_for_51 = pd.read_csv(PATH + 'train_51_prep.csv')
valid_for_51 = pd.read_csv(PATH + 'valid_51_prep.csv')
test_for_51 = pd.read_csv(PATH + 'test_51_prep.csv')

In [202]:
y_train_51 = train_for_51[target]
y_test_51 = valid_for_51[target]
y_val_51 = test_for_51[target]

train_for_51.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for_51.drop('Unnamed: 0', axis = 1, inplace = True)
test_for_51.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for_51.columns.to_list()
col.remove(target)


X_1_2_51 = train_for_51[col]
X_2_2_51 = valid_for_51[col]
X_3_2_51 = test_for_51[col]

In [21]:
import tensorflow as tf
tf.version.VERSION

'2.3.0'

In [61]:
#dd_all.shape

(217, 7)

In [62]:
#dd_best = dd_all

In [177]:
dd_best.shape

(17, 7)

In [178]:
new_col = ['Val_APS', 'Val_APS_conf_int', 'Val_APS_conf_l', 'Val_APS_conf_r', 'Val_GINI', 'Val_AUC', 'Val_AUC_conf_l', 
           'Val_AUC_conf_r',
          'TEST_APS', 'TEST_APS_conf_int', 'TEST_APS_conf_l', 'TEST_APS_conf_r', 'TEST_GINI', 'TEST_AUC', 'TEST_AUC_conf_int', 
           'TEST_AUC_conf_l', 'TEST_AUC_conf_r',
          'TRAIN_APS', 'TRAIN_GINI', 'Count_epoch', 'Count_feature', 'Min_loss', 'LR_schedule', 
           'OOT2013_APS', 'OOT2013_APS_conf', 'OOT2013_GINI', 'OOT2013_AUC', 'OOT2013_AUC_conf',
          'saved_money_test', 'saved_money_test_all', 'saved_money_test_check' ,
           'saved_money_val', 'saved_money_val_all', 'saved_money_val_check' ,
           'saved_money_oot', 
           'saved_money_test_cor' , 'saved_money_test_all_cor' , 'saved_money_test_all_check' ,
          'precision_test', 'recall_test', 'tp_test', 'tn_test',
          'precision_val', 'recall_val', 'tp_val', 'tn_val',
          'Val_APS_conf_int_1', 'TEST_APS_conf_int_1']

In [179]:
dd_best = dd_best.reindex(dd_best.columns.tolist() + new_col, axis=1)

In [180]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D
import tensorflow

In [181]:
""" 
blocks/one_d.py
Author: Ankit Gupta

Implementations of various DenseNet blocks for 1D sequences

This module contains helper functions that define the various subcomponents of a DenseNet. This includes dense blocks and transition blocks.

"""

#from tf.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D


def H_l(k, bottleneck_size, kernel_width):
    """ 
    A single convolutional "layer" as defined by Huang et al. Defined as H_l in the original paper
    
    :param k: int representing the "growth rate" of the DenseNet
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :param kernel_width: int representing the width of the main convolutional kernel
    :return a function wrapping the keras layers for H_l
    """

    use_bottleneck = bottleneck_size > 0
    num_bottleneck_output_filters = k * bottleneck_size

    def f(x):
        if use_bottleneck:
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = Conv1D(
                num_bottleneck_output_filters,
                1,
                strides=1,
                padding="same",
                dilation_rate=1)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            k,
            kernel_width,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        return x
    return f


def dense_block(k, num_layers, kernel_width, bottleneck_size):
    """
    A single dense block of the DenseNet
    
    :param k: int representing the "growth rate" of the DenseNet
    :param num_layers: int represending the number of layers in the block
    :param kernel_width: int representing the width of the main convolutional kernel
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :return a function wrapping the entire dense block
    """
    def f(x):
        layers_to_concat = [x]
        for _ in range(num_layers):
            x = H_l(k, bottleneck_size, kernel_width)(x)
            layers_to_concat.append(x)
            x = Concatenate(axis=-1)(layers_to_concat)
        return x
    return f


def transition_block(pool_size=2, stride=2, theta=0.5):
    """
    A single transition block of the DenseNet
    
    :param pool_size: int represending the width of the average pool
    :param stride: int represending the stride of the average pool
    :param theta: int representing the amount of compression in the 1x1 convolution. Set to 1 for no compression.
    :return a function wrapping the entire transition block
    """    
    assert theta > 0 and theta <= 1

    def f(x):
        num_transition_output_filters = int(int(x.shape[2]) * float(theta))
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            num_transition_output_filters,
            1,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        x = AveragePooling1D(
            pool_size=pool_size,
            strides=stride,
            padding="same")(x)
        return x
    return f


def DenseNet_1(
        k,
        block_sizes,
        conv_kernel_width,
        bottleneck_size,
        transition_pool_size,
        transition_pool_stride,
        theta,
        initial_conv_width,
        initial_stride,
        initial_filters,
        initial_pool_width,
        initial_pool_stride,
        use_global_pooling):
    def f(x):
        x = Conv1D(
            initial_filters,
            initial_conv_width,
            strides=initial_stride,
            padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = MaxPooling1D(
            pool_size=initial_pool_width,
            strides=initial_pool_stride,
            padding="same")(x)

        # Add all but the last dense block
        for block_size in block_sizes[:-1]:
            x = dense_block(
                k,
                block_size,
                conv_kernel_width,
                bottleneck_size)(x)
            x = transition_block(
                pool_size=transition_pool_size,
                stride=transition_pool_stride,
                theta=theta)(x)

        # Add the last dense block
        final_block_size = block_sizes[-1]
        x = dense_block(
            k,
            final_block_size,
            conv_kernel_width,
            bottleneck_size)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        if use_global_pooling:
            x = GlobalAveragePooling1D()(x)
        return x
    return f



class DenseNetCustom(tensorflow.keras.models.Model):
    """  
    Create a Keras Model Object that is an implementation of DenseNet with a custom number of parameters. The number of layers 
    per dense block can be specified by block_sizes.
    :param input_shape: The shape of the inputs without the batch dimension. This should be a valid 1D sequence, 
    such as (244, 25). 
    :param num_outputs: the number of classes to predict
    :param k: The "growth rate" of the DenseNet model
    :param block_sizes: A list of ints with the number of layers in each block. Example: [5, 10, 25, 17].
    :param conv_kernel_width: The kernel width of each convolution in the dense blocks.
    :param bottleneck_size: The size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :param transition_pool_size: pool_size in the transition layer
    :param transition_pool_stride: pooling stride in the transition layer
    :param theta: Amount of compression in the transition layer. Set to 1 for no compression.
    :param initial_conv_width: Kernel width for the one convolution before the dense blocks
    :param initial_stride: Stride for the one convolution before the dense blocks
    :param initial_filters: Number of filters for the one convolution before the dense blocks
    :param initial_pool_width: pool_size for the one pooling before the dense blocks
    :param initial_pool_stride: stride for the one pooling before the dense blocks 
    """
    def __init__(
            self,
            input_shape,
            num_outputs=1000,
            k=32,
            block_sizes=None,
            conv_kernel_width=3,
            bottleneck_size=4,
            transition_pool_size=2,
            transition_pool_stride=2,
            theta=0.5,
            initial_conv_width=7,
            initial_stride=2,
            initial_filters=64,
            initial_pool_width=3,
            initial_pool_stride=2):
        if not block_sizes:
            raise ValueError("block_sizes must be specified")
        model_input = Input(shape=input_shape)
        output = DenseNet_1(
            k,
            block_sizes,
            conv_kernel_width,
            bottleneck_size,
            transition_pool_size,
            transition_pool_stride,
            theta,
            initial_conv_width,
            initial_stride,
            initial_filters,
            initial_pool_width,
            initial_pool_stride,
            use_global_pooling=True)(model_input)
        output = Dense(num_outputs, activation="softmax")(output)
        super(DenseNetCustom, self).__init__(model_input, output)

In [450]:
spider_ord = pd.read_csv('spider_orderxlsx.csv', sep = ';')

In [451]:
dd_all = pd.merge(dd_best, spider_ord, how='left', on='name')

In [453]:
dd_all.sort_values('index', inplace=True)

In [70]:
dd_all.head()

,full_name,name,type,ctime_d,ctime,time_num,csv_name
118,model_spider6_gridTrue_0.0001_3_10_100.h5,model_spider6_gridTrue_0.0001_3_10_100,h5,1.619145e+09,23.04.2021 05:36:26,20210423,model_spider6_gridTrue_0.0001_3_10_100.csv
167,model_spider6_grid_dropconst_0.25True_0.0001_3...,model_spider6_grid_dropconst_0.25True_0.0001_3...,h5,1.622795e+09,04.06.2021 11:15:18,20210604,model_spider6_grid_dropconst_0.25True_0.0001_3...
170,model_spider6_grid_wo_dropconv_True_0.0001_3_1...,model_spider6_grid_wo_dropconv_True_0.0001_3_1...,h5,1.622659e+09,02.06.2021 21:34:29,20210602,model_spider6_grid_wo_dropconv_True_0.0001_3_1...
199,model_spider8_internal_grid_20_7_5_64_0.0002_2...,model_spider8_internal_grid_20_7_5_64_0.0002_2...,h5,1.621035e+09,15.05.2021 02:28:04,20210515,model_spider8_internal_grid_20_7_5_64_0.0002_2...
200,model_spider8_internal_grid_dropconst_0.2520_7...,model_spider8_internal_grid_dropconst_0.2520_7...,h5,1.622790e+09,04.06.2021 09:58:23,20210604,model_spider8_internal_grid_dropconst_0.2520_7...


In [480]:
_124_ord = pd.read_csv('124+target_wo_btest.csv', sep = ',')

In [73]:
#для 163
list_train_for = pd.read_csv(PATH + 'train_163_prep.csv', nrows = 10)
list_train_for.drop('Unnamed: 0', axis = 1, inplace = True)
col_train_163 = list_train_for.columns.to_list()
col_train_163.remove(target)
len(col_train_163)

163

In [74]:
#для 124
list_train_for = pd.read_csv('124+target_wo_btest.csv', sep = ',')
col_train_124 = list_train_for['var_wo_btest'].to_list()
col_train_124.remove(target)
len(col_train_124)

124

In [186]:
dd_all = dd_best
dd_all.shape

(17, 55)

In [197]:
def antifraud_profit_bytt3(data, prediction, loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[data[period_name] == period].copy()
        
        data1.sort_values(by = prediction, ascending = False, inplace = True)
        data1.reset_index(drop = True, inplace = True)
        
        data2 = data1.head(to_select)
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            #добавила .loc[(data[period_name] == period)] иначе учитвыются вхождения за 13 недель
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]   
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR',  prediction, loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    for period in periods:
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0] , count_check #- список ТТ

In [198]:
def antifraud_profit_bytt_cor(data,  prediction, loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    tab_all_tt = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[(data[period_name] == period) & (~data[tt_code].isin(all_fraud_tt))].copy()
        
        data1.sort_values(by = prediction, ascending = False, inplace = True)
        data1.reset_index(drop = True, inplace = True)
        
        data2 = data1.head(to_select)
            
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]  
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR', loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    k = 0
    for period in periods:
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
        k +=1
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0],  len(final_fraud_dict), count_check #- список ТТ

In [211]:
#бизнес-метрика: на каждом периоде отбираем ТОП-ТТ и считаем по ним убыток
#по данным из AF, в неделю проверяется 40 ТТ
#поэтому для oot за 2013г берется 40, для test/ val берется 5, т.к. test/ val - это пот 10% от выборки
#scores_optuna['saved_money_test_multiply'] = [saved_money_test_only_1*5] - 5 - это коэффициент, 
#который приводит результат к полным данным, для test/ val нужно брать 10


for j in dd_all.index: #для стандартных наборов dd_all['index']: #
    
    
    mod = dd_all['full_name'][j]
    tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
    print (j, mod, 'begin', '| time: ' , tm)
    
    if mod.find('_dense_') >= 0:
        model_X = tf.keras.models.load_model(('C:/Users/asmirnova5/Desktop/Python-scripts/Python-notebook/depp/' +mod),
                                             custom_objects={'DenseNetCustom':DenseNetCustom})
    else:
        model_X = load_model('C:/Users/asmirnova5/Desktop/Python-scripts/Python-notebook/depp/' +mod)
    
    if str(dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0]) != 'nan':
        history_XX = pd.read_csv('C:/Users/asmirnova5/Desktop/Python-scripts/Python-notebook/depp/' + 
                                 dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0])
    
    test = pd.DataFrame.from_dict(model_X.get_config()['layers'])
    
    col_train =[]
    
    if test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == 163:
        col_train = col_train_163
    elif test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == 124:
        col_train = col_train_124
    
    
    #"""
    if test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == len(col_train):
        #predict_class_val = model_X.predict(X_2_2_163)
        #dd_all.at[j, 'Val_APS'] = metrics.average_precision_score(y_test_163, predict_class_val[:,1])
        #dd_all.at[j, 'Val_GINI']  = 2*metrics.roc_auc_score(y_test_163, predict_class_val[:,1]) - 1
        
        X_2 = X_2_2.copy(deep=True)   
        X_2['predict_class'] = model_X.predict(X_2[col_train])[:,1]
        X_2 = pd.concat( [X_2, y_test] , axis = 1 )

        dd_all.at[j, 'Val_APS'] = metrics.average_precision_score(y_test, X_2['predict_class'])
        dd_all.at[j, 'Val_GINI']  = 2*metrics.roc_auc_score(y_test, X_2['predict_class']) - 1
        
        money, tt_all, tt_uniq = antifraud_profit_bytt3(X_2, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        
        dd_all.at[j, 'saved_money_val'] = money
        dd_all.at[j, 'saved_money_val_all'] = tt_all
        dd_all.at[j, 'saved_money_val_check'] = tt_uniq
        
        #####
        ci_a = aps_ci_logit(X_2['BAD_FLAG'], X_2['predict_class'])
        dd_all.at[j,'Val_APS_conf_l'] = ci_a[0]
        dd_all.at[j,'Val_APS_conf_r'] = ci_a[1]
        dd_all.at[j,'Val_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        
        
        ci_a_1 = aps_ci_logit(X_2['BAD_FLAG'], X_2['predict_class'], epsilon = 0.01)
        dd_all.at[j,'Val_APS_conf_int_1'] = (ci_a_1[1] - ci_a_1[0])/2
        
        auc, auc_var, ci = auc_ci_Delong(y_true=X_2['BAD_FLAG'], y_scores=X_2['predict_class'])
        dd_all.at[j,'Val_AUC'] = auc
        dd_all.at[j,'Val_AUC_conf_l'] = ci[0]
        dd_all.at[j,'Val_AUC_conf_r'] = ci[1]
        dd_all.at[j,'Val_AUC_conf_int'] = (ci[1] - ci[0])/2
        
        X_2['predict'] = np.round(X_2['predict_class'])
        dd_all.at[j, 'precision_val'] = precision_score(y_test, X_2['predict'])
        dd_all.at[j, 'recall_val'] = recall_score(y_test, X_2['predict'])
        tn, fp, fn, tp = confusion_matrix(y_test, X_2['predict']).ravel()
        dd_all.at[j, 'tp_val'] = tp
        dd_all.at[j, 'tn_val'] = tn        
        
        del X_2
        #print(dd_all['Val_APS'][j] , dd_all['Val_GINI'][j], )

        ####### test
        
        X_3 = X_3_2.copy(deep=True)   
        X_3['predict_class'] = model_X.predict(X_3[col_train])[:,1]
        X_3 = pd.concat( [X_3, y_val] , axis = 1 )
        
        dd_all.at[j, 'TEST_APS'] = metrics.average_precision_score(y_val, X_3['predict_class'])
        dd_all.at[j, 'TEST_GINI']  = 2*metrics.roc_auc_score(y_val, X_3['predict_class']) - 1
        
        money, tt_all, tt_uniq = antifraud_profit_bytt3(X_3, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        
        dd_all.at[j, 'saved_money_test'] = money
        dd_all.at[j, 'saved_money_test_all'] = tt_all
        dd_all.at[j, 'saved_money_test_check'] = tt_uniq
        
        ci_a = aps_ci_logit(X_3['BAD_FLAG'], X_3['predict_class'])
        dd_all.at[j,'TEST_APS_conf_l'] = ci_a[0]
        dd_all.at[j,'TEST_APS_conf_r'] = ci_a[1]
        
        ci_a_1 = aps_ci_logit(X_3['BAD_FLAG'], X_3['predict_class'], epsilon = 0.01)
        dd_all.at[j,'TEST_APS_conf_int_1'] = (ci_a_1[1] - ci_a_1[0])/2
        
        dd_all.at[j,'TEST_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        auc, auc_var, ci = auc_ci_Delong(y_true=X_3['BAD_FLAG'], y_scores=X_3['predict_class'])
        dd_all.at[j,'TEST_AUC'] = auc
        dd_all.at[j,'TEST_AUC_conf_l'] = ci[0]
        dd_all.at[j,'TEST_AUC_conf_r'] = ci[1]
        dd_all.at[j,'TEST_AUC_conf_int'] = (ci[1] - ci[0])/2
        
        X_3['predict'] = np.round(X_3['predict_class'])
        dd_all.at[j, 'precision_test'] = precision_score(y_val, X_3['predict'])
        dd_all.at[j, 'recall_test'] = recall_score(y_val, X_3['predict'])
        tn, fp, fn, tp = confusion_matrix(y_val, X_3['predict']).ravel()
        dd_all.at[j, 'tp_test'] = tp
        dd_all.at[j, 'tn_test'] = tn 
        
        sums_ , tab_ , fraud_uniq_, count_check = antifraud_profit_bytt_cor(X_3, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        
        
        dd_all.at[j,'saved_money_test_cor'] = sums_
        dd_all.at[j,'saved_money_test_all_cor'] = tab_
        dd_all.at[j, 'saved_money_test_all_check'] = count_check
        
        del X_3
        
        

    ####
    dd_all.at[j, 'Count_epoch'] = history_XX.shape[0]
    dd_all.at[j, 'Min_loss'] = min(history_XX['val_loss'])
    dd_all.at[j, 'Count_feature'] = test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1]
    
    if 'lr' in history_XX.columns:
        dd_all.at[j, 'LR_schedule'] = len(np.unique(history_XX['lr']))
        
    dd_all.to_excel('BEST_Privat_check_20210909.xlsx')
                                                                        
    #tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
    #print(j, mod, 'end', '| time: ' , tm)

186 model_spider8_internal_grid_15_3_10_100_0.0002_200512_8.h5 begin | time:  09.09.2021 12:26:43


In [216]:
val_c = X_2_2['PERIOD_7'].value_counts()

In [218]:
test_c = X_3_2['PERIOD_7'].value_counts()

In [219]:
train_c = X_1_2['PERIOD_7'].value_counts()

In [220]:
all_c = pd.concat([val_c, test_c, train_c], axis = 1)

In [221]:
len(val_c), len(test_c), len(all_c)

(282, 290, 292)

In [223]:
all_c.to_csv('group_by_period_7.csv')

In [200]:
dd_all

,full_name,name,type,ctime_d,ctime,time_num,csv_name,Val_APS,Val_APS_conf_int,Val_APS_conf_l,...,recall_test,tp_test,tn_test,precision_val,recall_val,tp_val,tn_val,Val_APS_conf_int_1,TEST_APS_conf_int_1,Val_AUC_conf_int
23,model_cnn_internal_grid_False_3_10_100_8_20050...,model_cnn_internal_grid_False_3_10_100_8_200507_7,h5,1.620363e+09,07.05.2021 07:52:15,20210507,model_cnn_internal_grid_False_3_10_100_8_20050...,0.056678,0.001053,0.055635,...,0.324373,181.0,184454.0,0.068232,0.373913,215.0,181889.0,0.001383,0.001526,0.010637
24,model_cnn_internal_grid_False_5_10_30_6_200505...,model_cnn_internal_grid_False_5_10_30_6_200505_13,h5,1.620212e+09,05.05.2021 13:51:10,20210505,model_cnn_internal_grid_False_5_10_30_6_200505...,0.055569,0.001043,0.054535,...,0.241935,135.0,185378.0,0.072072,0.236522,136.0,183074.0,0.001371,0.001449,0.009897
30,model_cnn_internal_grid_False_5_15_60_6_200507...,model_cnn_internal_grid_False_5_15_60_6_200507_20,h5,1.620407e+09,07.05.2021 20:04:52,20210507,model_cnn_internal_grid_False_5_15_60_6_200507...,0.067691,0.001144,0.066557,...,0.032258,18.0,186849.0,0.126761,0.046957,27.0,184639.0,0.001503,0.001460,0.010616
31,model_cnn_internal_grid_False_5_15_60_8_200508...,model_cnn_internal_grid_False_5_15_60_8_200508_11,h5,1.620463e+09,08.05.2021 11:36:06,20210508,model_cnn_internal_grid_False_5_15_60_8_200508...,0.064010,0.001114,0.062905,...,0.141577,79.0,186309.0,0.106901,0.137391,79.0,184165.0,0.001464,0.001487,0.013482
38,model_cnn_internal_grid_False_7_15_60_3_200502...,model_cnn_internal_grid_False_7_15_60_3_200502_12,h5,1.619949e+09,02.05.2021 12:44:32,20210502,model_cnn_internal_grid_False_7_15_60_3_200502...,0.061886,0.001097,0.060798,...,0.387097,216.0,183688.0,0.062500,0.415652,239.0,181240.0,0.001441,0.001327,0.007233
42,model_cnn_internal_grid_False_7_5_60_3_200501_...,model_cnn_internal_grid_False_7_5_60_3_200501_13,h5,1.619864e+09,01.05.2021 13:12:41,20210501,model_cnn_internal_grid_False_7_5_60_3_200501_...,0.055998,0.001047,0.054960,...,0.211470,118.0,185610.0,0.073860,0.222609,128.0,183220.0,0.001376,0.001330,0.009160
50,model_dense_PrivatData_c3c3_k10_t0.5_w7.h5,model_dense_PrivatData_c3c3_k10_t0.5_w7,h5,1.620824e+09,12.05.2021 15:58:19,20210512,model_dense_PrivatData_c3c3_k10_t0.5_w7.csv,0.052754,0.001018,0.051746,...,0.148746,83.0,185924.0,0.087239,0.210435,121.0,183559.0,0.001337,0.001244,0.009610
63,model_dense_PrivatData_c4c4_k5_t0.5_w3.h5,model_dense_PrivatData_c4c4_k5_t0.5_w3,h5,1.620736e+09,11.05.2021 15:18:58,20210511,model_dense_PrivatData_c4c4_k5_t0.5_w3.csv,0.060424,0.001085,0.059349,...,0.102151,57.0,186648.0,0.135065,0.090435,52.0,184492.0,0.001425,0.001509,0.008230
67,model_dense_PrivateData_c3c3_k10_t0.5_w5.h5,model_dense_PrivateData_c3c3_k10_t0.5_w5,h5,1.620736e+09,11.05.2021 15:18:58,20210511,model_dense_PrivateData_c3c3_k10_t0.5_w5.csv,0.059045,0.001073,0.057981,...,0.157706,88.0,186163.0,0.094533,0.144348,83.0,184030.0,0.001410,0.001463,0.009084
73,model_fDenseNet6_PrivateData_with_B_tests_0.00...,model_fDenseNet6_PrivateData_with_B_tests_0.00...,h5,1.620824e+09,12.05.2021 15:58:19,20210512,model_fDenseNet6_PrivateData_with_B_tests_0.00...,0.049400,0.000986,0.048423,...,0.182796,102.0,186044.0,0.072745,0.130435,75.0,183869.0,0.001296,0.001439,0.008881


In [127]:
X_3 = X_3_2.copy(deep=True)   
X_3['predict_class'] = model_X.predict(X_3[col_train])[:,1]
X_3 = pd.concat( [X_3, y_val] , axis = 1 )

In [128]:
X_44 = X_3[X_3['BAD_FLAG'] == 1]

In [129]:
X_44.shape

(558, 169)

In [141]:
def antifraud_profit_max(data, prediction, loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    tab_all_tt = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[(data[period_name] == period) & (data[target] == 1)].copy()
        data1.reset_index(drop = True, inplace = True)
        
        if to_select < data1.shape[0]:
            data2 = data1.head(to_select)
        else:
            data2 = data1
            
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        print(period, data2.shape[0], len(fraud_tt))
        
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]  
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR',  prediction, loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    k = 0
    for period in periods:
        print(k, sep = '_')
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
        k +=1
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0], tab_loss, len(final_fraud_dict), count_check #- список ТТ

In [152]:
def antifraud_profit_max_cor(data,  loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    tab_all_tt = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[(data[period_name] == period) & (data[target] == 1)& (~data[tt_code].isin(all_fraud_tt))].copy()
        data1.reset_index(drop = True, inplace = True)
        
        if to_select < data1.shape[0]:
            data2 = data1.head(to_select)
        else:
            data2 = data1
            
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        print(period, data2.shape[0], len(fraud_tt))
        
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]  
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR', loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    k = 0
    for period in periods:
        print(k, sep = '_')
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
        k +=1
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0], tab_loss, len(final_fraud_dict), count_check #- список ТТ

In [154]:
sums_only, count_all_tt, all_tt, count_uniq_tt, count_check = antifraud_profit_max_cor(X_3,   
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)

0
18.0 0 0
1
19.0 0 0
2
20.0 0 0
3
21.0 0 0
4
22.0 1 1
5
23.0 0 0
6
24.0 0 0
7
25.0 0 0
8
26.0 0 0
9
27.0 0 0
10
28.0 0 0
11
29.0 0 0
12
30.0 0 0
13
31.0 0 0
14
32.0 0 0
15
33.0 0 0
16
34.0 0 0
17
35.0 0 0
18
36.0 0 0
19
37.0 0 0
20
38.0 0 0
21
39.0 0 0
22
40.0 0 0
23
41.0 0 0
24
42.0 1 1
25
43.0 0 0
26
44.0 0 0
27
45.0 1 1
28
46.0 1 1
29
47.0 0 0
30
48.0 0 0
31
49.0 0 0
32
50.0 0 0
33
51.0 0 0
34
52.0 0 0
35
53.0 0 0
36
54.0 1 1
37
55.0 0 0
38
56.0 0 0
39
57.0 0 0
40
58.0 0 0
41
59.0 0 0
42
60.0 0 0
43
61.0 0 0
44
62.0 0 0
45
63.0 0 0
46
64.0 0 0
47
65.0 0 0
48
66.0 0 0
49
67.0 0 0
50
68.0 0 0
51
69.0 0 0
52
70.0 0 0
53
71.0 0 0
54
72.0 0 0
55
73.0 0 0
56
74.0 0 0
57
75.0 0 0
58
76.0 0 0
59
77.0 0 0
60
78.0 0 0
61
79.0 0 0
62
80.0 0 0
63
81.0 0 0
64
82.0 0 0
65
83.0 0 0
66
84.0 0 0
67
85.0 0 0
68
86.0 0 0
69
87.0 1 1
70
88.0 0 0
71
89.0 1 1
72
90.0 0 0
73
91.0 0 0
74
92.0 0 0
75
93.0 0 0
76
94.0 0 0
77
95.0 0 0
78
96.0 0 0
79
97.0 0 0
80
98.0 0 0
81
99.0 0 0
82
100.0 0 0
83
101.0 0 0


In [155]:
sums_only, count_all_tt, count_uniq_tt, count_check

(34111993.080299005, 134, 103, 134)

In [158]:
def antifraud_profit_rand_cor(data,  loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    tab_all_tt = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[(data[period_name] == period) & (~data[tt_code].isin(all_fraud_tt))].copy()
        data1 = data1.iloc[np.random.permutation(len(data1))]
        
        data2 = data1.head(to_select)
            
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        print(period, data2.shape[0], len(fraud_tt))
        
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]  
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR', loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    k = 0
    for period in periods:
        print(k, sep = '_')
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
        k +=1
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0], tab_loss, len(final_fraud_dict), count_check #- список ТТ

In [159]:
sums_only, count_all_tt, all_tt, count_uniq_tt, count_check = antifraud_profit_rand_cor(X_3,   
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)

0
18.0 5 0
1
19.0 5 0
2
20.0 5 0
3
21.0 5 0
4
22.0 5 0
5
23.0 5 0
6
24.0 5 0
7
25.0 5 0
8
26.0 5 0
9
27.0 5 0
10
28.0 5 0
11
29.0 5 0
12
30.0 5 0
13
31.0 5 0
14
32.0 5 0
15
33.0 5 0
16
34.0 5 0
17
35.0 5 0
18
36.0 5 0
19
37.0 5 0
20
38.0 5 0
21
39.0 5 0
22
40.0 5 0
23
41.0 5 0
24
42.0 5 0
25
43.0 5 0
26
44.0 5 0
27
45.0 5 0
28
46.0 5 0
29
47.0 5 0
30
48.0 5 0
31
49.0 5 0
32
50.0 5 0
33
51.0 5 0
34
52.0 5 0
35
53.0 5 0
36
54.0 5 0
37
55.0 5 0
38
56.0 5 0
39
57.0 5 0
40
58.0 5 0
41
59.0 5 0
42
60.0 5 0
43
61.0 5 0
44
62.0 5 0
45
63.0 5 0
46
64.0 5 0
47
65.0 5 0
48
66.0 5 0
49
67.0 5 0
50
68.0 5 0
51
69.0 5 0
52
70.0 5 0
53
71.0 5 0
54
72.0 5 0
55
73.0 5 0
56
74.0 5 0
57
75.0 5 0
58
76.0 5 0
59
77.0 5 0
60
78.0 5 0
61
79.0 5 0
62
80.0 5 0
63
81.0 5 0
64
82.0 5 0
65
83.0 5 0
66
84.0 5 0
67
85.0 5 0
68
86.0 5 0
69
87.0 5 0
70
88.0 5 0
71
89.0 5 0
72
90.0 5 0
73
91.0 5 0
74
92.0 5 0
75
93.0 5 0
76
94.0 5 0
77
95.0 5 0
78
96.0 5 0
79
97.0 5 0
80
98.0 5 0
81
99.0 5 0
82
100.0 5 0
83
101.0 5 0


In [161]:
sums_only, count_all_tt,  count_uniq_tt, count_check

(3824205.96125, 11, 11, 1403)

In [162]:
def antifraud_profit_rand(data,  loss, target, period_name, 
                                                   tt_code, to_select = 30, thre=13):
    #для val/ test to_select = 5
    #для train - 30
    #для 2013г - 40
    #thre=13 - на сколько недель выключается ТТ
    
#     all_fraud_tt = []
    sum_losses1 = []
    
    all_fraud_dict = {}
    final_fraud_dict = {}
    tab_loss = pd.DataFrame()
    tab_all_tt = pd.DataFrame()
    count_check = 0
    
    def antifraud_profit_metrics_for_period_unique_tt(period):
        
        nonlocal all_fraud_dict
        nonlocal sum_losses1
        nonlocal final_fraud_dict
        nonlocal tab_loss
        nonlocal count_check
        
        all_fraud_dict = {k:v for k, v in all_fraud_dict.items() if (period-v)< thre}
        all_fraud_tt = list(all_fraud_dict.keys())
        
        data1 = data.loc[(data[period_name] == period)].copy()
        data1 = data1.iloc[np.random.permutation(len(data1))]
        
        data2 = data1.head(to_select)
            
        count_check += data2.shape[0]
        
        fraud_tt = list(set(data2.loc[(data2[target] == 1) & (~data2[tt_code].isin(all_fraud_tt)), tt_code].to_list()))
        print(period, data2.shape[0], len(fraud_tt))
        
        if len(fraud_tt) > 0:
            fraud_tt_check = [i in all_fraud_tt for i in fraud_tt]
            assert max(fraud_tt_check) == False
        for tt in fraud_tt:
            all_fraud_dict[tt]=period
            if tt not in final_fraud_dict.keys():
                final_fraud_dict[tt] = [period]
            else:
                final_fraud_dict[tt] += [period]
        
        losses_data = data.loc[(data[target] == 1) & 
                               (data[tt_code].isin(fraud_tt)&
                                (data[period_name] >= period)&
                                (data[period_name] < period+thre))]
        #print(losses_data.shape)
        
        if losses_data.shape[0] > 0:
            ll = losses_data.copy(deep=True).loc[(data[period_name] == period)]  
            ll.loc[:, 'period'] = period
            ll = ll[[tt_code, period_name, 'MONTH_YEAR', loss, target, 'period']]
            
            if tab_loss.shape[0] == 0:
                tab_loss = ll
            else:
                tab_loss = pd.concat([tab_loss, ll], axis = 0)
            
        
        sum_loss1 = sum(losses_data.groupby(tt_code)[loss].max())
        sum_losses1.append(sum_loss1)
        gc.collect()
        
    
    periods = sorted(data[period_name].unique())
        
    par_res = []
    k = 0
    for period in periods:
        print(k, sep = '_')
        par_res.append(antifraud_profit_metrics_for_period_unique_tt(period))
        k +=1
    
    sums_only1 = sum(sum_losses1)
    
    return sums_only1 , tab_loss.shape[0], tab_loss, len(final_fraud_dict), count_check #- список ТТ

In [163]:
sums_only, count_all_tt, all_tt, count_uniq_tt, count_check = antifraud_profit_rand(X_3,   
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
sums_only, count_all_tt, count_uniq_tt, count_check

0
18.0 5 0
1
19.0 5 0
2
20.0 5 0
3
21.0 5 0
4
22.0 5 0
5
23.0 5 0
6
24.0 5 0
7
25.0 5 0
8
26.0 5 0
9
27.0 5 0
10
28.0 5 0
11
29.0 5 0
12
30.0 5 0
13
31.0 5 0
14
32.0 5 0
15
33.0 5 0
16
34.0 5 0
17
35.0 5 0
18
36.0 5 0
19
37.0 5 0
20
38.0 5 0
21
39.0 5 0
22
40.0 5 0
23
41.0 5 0
24
42.0 5 0
25
43.0 5 0
26
44.0 5 0
27
45.0 5 0
28
46.0 5 0
29
47.0 5 0
30
48.0 5 0
31
49.0 5 0
32
50.0 5 0
33
51.0 5 0
34
52.0 5 0
35
53.0 5 0
36
54.0 5 0
37
55.0 5 0
38
56.0 5 0
39
57.0 5 0
40
58.0 5 0
41
59.0 5 0
42
60.0 5 0
43
61.0 5 0
44
62.0 5 0
45
63.0 5 0
46
64.0 5 0
47
65.0 5 0
48
66.0 5 0
49
67.0 5 0
50
68.0 5 0
51
69.0 5 0
52
70.0 5 0
53
71.0 5 0
54
72.0 5 0
55
73.0 5 0
56
74.0 5 0
57
75.0 5 0
58
76.0 5 0
59
77.0 5 0
60
78.0 5 0
61
79.0 5 0
62
80.0 5 0
63
81.0 5 0
64
82.0 5 0
65
83.0 5 0
66
84.0 5 0
67
85.0 5 0
68
86.0 5 0
69
87.0 5 0
70
88.0 5 0
71
89.0 5 0
72
90.0 5 0
73
91.0 5 0
74
92.0 5 0
75
93.0 5 0
76
94.0 5 0
77
95.0 5 0
78
96.0 5 0
79
97.0 5 0
80
98.0 5 0
81
99.0 5 0
82
100.0 5 0
83
101.0 5 0


(2259155.2459279997, 6, 6, 1409)

In [143]:
sums_only, count_all_tt, count_uniq_tt, count_check

(32328836.945823006, 111, 83, 381)

In [140]:
sums_only, count_all_tt, count_uniq_tt, count_check

(32328836.945823006, 445, 83, 381)

In [130]:
sum(X_44['LOSS_90P'])

159582048.00125313

In [131]:
np.median(X_44['LOSS_90P'])

115951.42285999999

In [125]:
sum(X_3_2[X_3_2])

2

In [126]:
y_val.value_counts()

0    186942
1       558
Name: BAD_FLAG, dtype: int64

In [148]:
X_3 = X_3_2.copy(deep=True)   
X_3 = pd.concat( [X_3, y_val] , axis = 1 )

In [425]:
table = pd.pivot_table(X_3, values=target,
                    index=['MONTH_YEAR'], aggfunc=[np.sum, 'count'])
table

,sum,count
,BAD_FLAG,BAD_FLAG
MONTH_YEAR,,
201404.0,0,3
201405.0,4,4
201406.0,0,15
201407.0,9,546
201408.0,93,1548
...,...,...
201906.0,0,4435
201907.0,0,3633


In [427]:
table.to_csv('stat_bad_test.csv')

In [274]:
#бизнес-метрика: на каждом периоде отбираем ТОП-ТТ и считаем по ним убыток
#по данным из AF, в неделю проверяется 40 ТТ
#поэтому для oot за 2013г берется 40, для test/ val берется 5, т.к. test/ val - это пот 10% от выборки
#scores_optuna['saved_money_test_multiply'] = [saved_money_test_only_1*5] - 5 - это коэффициент, 
#который приводит результат к полным данным, для test/ val нужно брать 10


for j in dd_all.index:
    
    
    mod = dd_all['full_name'][j]
    if mod.find('_dense_') >= 0:
        model_X = tf.keras.models.load_model(('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' +mod),
                                             custom_objects={'DenseNetCustom':DenseNetCustom})
    else:
        model_X = load_model('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' +mod)
    
    if str(dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0]) != 'nan':
        history_XX = pd.read_csv('C:/Users/safanasev/Desktop/Python-scripts/Python-notebook/depp/' + 
                                 dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0])
    
    test = pd.DataFrame.from_dict(model_X.get_config()['layers'])
    
    dd_all.at[j, 'Conv' ] = sum(test['class_name']=='Conv1D')
    
    dd_all.at[j, 'Conv_kernel_size'] = test[test['class_name'] =='Conv1D']['config'][1]['kernel_size'][0]
    dd_all.at[j, 'Conv_kernel_reg'] = test[test['class_name'] =='Conv1D']['config'][1]['kernel_regularizer']
    
    c = []
    for i in test[test['class_name'] =='Conv1D'].index:
        c.append(test[test['class_name'] =='Conv1D']['config'][i]['filters'])
    dd_all.at[j, 'Conv_filter' ] = c
    
    dd_all.at[j, 'Dense' ] = sum(test['class_name']=='Dense')
    
    k = []
    k_reg = []
    for i in test[test['class_name'] =='Dense'].index:
        k.append(test[test['class_name'] =='Dense']['config'][i]['units'])
        k_reg.append(test[test['class_name'] =='Dense']['config'][i]['kernel_regularizer'])
        
    dd_all.at[j, 'Dense_shape' ] = k  
    dd_all.at[j, 'Dropupt' ] = sum(test['class_name']=='Dropout')
    dd_all.at[j, 'Dense_reg' ] = k_reg
    
    b_reg = []
    for i in test[test['class_name'] =='BatchNormalization'].index:
        b_reg.append(test[test['class_name'] =='BatchNormalization']['config'][i]['beta_regularizer']['config'])
        
    dd_all.at[j, 'BatchNorm_reg' ] = b_reg
    
    if sum(test['class_name']=='Dropout') > 0:
        do = []
        for i in test[test['class_name'] =='Dropout'].index:
            do.append(test[test['class_name'] =='Dropout']['config'][i]['rate'])
        #i = test[test['class_name'] =='Dropout'].index[0]
        dd_all.at[j, 'Dropupt_rate' ] = do
        
    dd_all.at[j, 'Opt_name'] = model_X.optimizer.get_config()['name']
    dd_all.at[j, 'Opt_Rate'] = model_X.optimizer.get_config()['learning_rate']
    
    
    #"""
    if test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == 163:
        
        X_3 = X_3_2.copy(deep=True)   
        X_3['predict_class'] = model_X.predict(X_3[col_train])[:,1]
        X_3 = pd.concat( [X_3, y_val] , axis = 1 )

        dd_all.at[j, 'Val_APS'] = metrics.average_precision_score(y_val, X_3['predict_class'])
        dd_all.at[j, 'Val_GINI']  = 2*metrics.roc_auc_score(y_val, X_3['predict_class']) - 1
        
        dd_all.at[j, 'saved_money_val'] = antifraud_profit_metrics_by_periods_unique_tt3(X_3, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        
        #####
        ci_a = aps_ci_logit(X_3['BAD_FLAG'], X_3['predict_class'])
        dd_all.at[j,'Val_APS_conf'] = ci_a
        dd_all.at[j,'Val_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        auc, auc_var, ci = auc_ci_Delong(y_true=X_3['BAD_FLAG'], y_scores=X_3['predict_class'])
        dd_all.at[j,'Val_AUC'] = auc
        dd_all.at[j,'Val_AUC_conf'] = ci
        dd_all.at[j,'Val_AUC_conf_int'] = (ci[1] - ci[0])/2
        del X_3
        print(dd_all['Val_APS'][j] , dd_all['Val_GINI'][j])

        ####### test
        
        X_2 = X_2_2.copy(deep=True)   
        X_2['predict_class'] = model_X.predict(X_2[col_train])[:,1]
        X_2 = pd.concat( [X_2, y_test] , axis = 1 )
        
        dd_all.at[j, 'TEST_APS'] = metrics.average_precision_score(y_test, X_2['predict_class'])
        dd_all.at[j, 'TEST_GINI']  = 2*metrics.roc_auc_score(y_test, X_2['predict_class']) - 1
        dd_all.at[j, 'saved_money_test'] = antifraud_profit_metrics_by_periods_unique_tt3(X_2, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        
        ci_a = aps_ci_logit(X_2['BAD_FLAG'], X_2['predict_class'])
        dd_all.at[j,'TEST_APS_conf'] = ci_a
        dd_all.at[j,'TEST_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        auc, auc_var, ci = auc_ci_Delong(y_true=X_2['BAD_FLAG'], y_scores=X_2['predict_class'])
        dd_all.at[j,'TEST_AUC'] = auc
        dd_all.at[j,'TEST_AUC_conf'] = ci
        dd_all.at[j,'TEST_AUC_conf_int'] = (ci[1] - ci[0])/2
        del X_2
        
        ### train
        predict_class_train = model_X.predict(X_1_2[col_train])
        dd_all.at[j, 'TRAIN_APS'] = metrics.average_precision_score(y_train, predict_class_train[:,1])
        dd_all.at[j, 'TRAIN_GINI']  = 2*metrics.roc_auc_score(y_train, predict_class_train[:,1]) - 1
        
        ## oot
        X_4 = X_4_2.copy(deep=True)   
        X_4['predict_class'] = model_X.predict(X_4[col_train])[:,1]
        X_4 = pd.concat( [X_4, y_oot] , axis = 1 )
        
        dd_all.at[j, 'OOT2013_APS'] = metrics.average_precision_score(y_oot, X_4['predict_class'])
        dd_all.at[j, 'OOT2013_GINI']  = 2*metrics.roc_auc_score(y_oot, X_4['predict_class']) - 1
        dd_all.at[j, 'saved_money_oot'] = antifraud_profit_metrics_by_periods_unique_tt3(X_4, 'predict_class', 
                                                            'LOSS_90P','BAD_FLAG', 'PERIOD_7','APPPOSID', 5, 13)
        ###
        ci_a = aps_ci_logit(X_4['BAD_FLAG'], X_4['predict_class'])
        dd_all.at[j,'OOT2013_APS_conf'] = ci_a
        dd_all.at[j,'OOT2013_APS_conf_int'] = (ci_a[1] - ci_a[0])/2
        
        auc, auc_var, ci = auc_ci_Delong(y_true=X_4['BAD_FLAG'], y_scores=X_4['predict_class'])
        dd_all.at[j,'OOT2013_AUC'] = auc
        dd_all.at[j,'OOT2013_AUC_conf'] = ci
        dd_all.at[j,'OOT2013_AUC_conf_int'] = (ci[1] - ci[0])/2
        del X_4
        
        
    if test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1] == 51:
        predict_class_val = model_X.predict(X_2_2_51)
        dd_all.at[j, 'Val_APS'] = metrics.average_precision_score(y_test_51, predict_class_val[:,1])
        dd_all.at[j, 'Val_GINI']  = 2*metrics.roc_auc_score(y_test_51, predict_class_val[:,1]) - 1

        predict_class_test = model_X.predict(X_3_2_51)
        dd_all.at[j, 'TEST_APS'] = metrics.average_precision_score(y_val_51, predict_class_test[:,1])
        dd_all.at[j, 'TEST_GINI']  = 2*metrics.roc_auc_score(y_val_51, predict_class_test[:,1]) - 1


        predict_class_train = model_X.predict(X_1_2_51)
        dd_all.at[j, 'TRAIN_APS'] = metrics.average_precision_score(y_train_51, predict_class_train[:,1])
        dd_all.at[j, 'TRAIN_GINI']  = 2*metrics.roc_auc_score(y_train_51, predict_class_train[:,1]) - 1
    #"""         

    ####
    dd_all.at[j, 'Count_epoch'] = history_XX.shape[0]
    dd_all.at[j, 'Min_loss'] = min(history_XX['val_loss'])
    dd_all.at[j, 'Count_feature'] = test[test['class_name'] =='InputLayer']['config'][0]['batch_input_shape'][1]
    
    if 'lr' in history_XX.columns:
        dd_all.at[j, 'LR_schedule'] = len(np.unique(history_XX['lr']))
                                                                        
    
    print(j, mod)

0.07078524813984674 0.8554926893738035
15 model_cnn_internal_grid_False_3_10_100_8_200507_7.h5
0.06438879706892832 0.8457897105609471
22 model_cnn_internal_grid_False_5_15_60_6_200507_20.h5
0.0527477443661799 0.8090982371662321
34 model_cnn_internal_grid_False_7_5_60_3_200501_13.h5
0.0801985352588117 0.8963679110945764
65 model_spider6_gridTrue_0.0001_3_10_64.h5
0.06938263227648486 0.8615555017179153
87 model_spider6_gridTrue_1e-05_5_5_64.h5


TypeError: 'NoneType' object is not subscriptable

In [259]:
dd_all['Val_APS'][15]

0.07078524813984674

In [264]:
dd_all

,full_name,name,type,ctime_d,ctime,time_num,csv_name,Conv,Conv_filter,Conv_kernel_size,...,saved_money_test,saved_money_val,saved_money_oot,Val_APS_conf_int,TEST_APS_conf_int,OOT2013_APS_conf_int,TEST_AUC_midle,TEST_AUC_conf_int,Val_AUC_conf_int,OOT2013_AUC_conf_int
15,model_cnn_internal_grid_False_3_10_100_8_20050...,model_cnn_internal_grid_False_3_10_100_8_200507_7,h5,1.620363e+09,07.05.2021 07:52:15,20210507,model_cnn_internal_grid_False_3_10_100_8_20050...,8,"[10, 20, 30, 40, 50, 60, 70, 80]",3,...,1.00849e+07,1.45528e+07,7.09358e+06,0.00116089,0.00105257,0.00155663,None,0.0106373,0.009605,0.011429
22,model_cnn_internal_grid_False_5_15_60_6_200507...,model_cnn_internal_grid_False_5_15_60_6_200507_20,h5,1.620407e+09,07.05.2021 20:04:52,20210507,model_cnn_internal_grid_False_5_15_60_6_200507...,6,"[15, 30, 45, 60, 75, 90]",5,...,9.82957e+06,1.51375e+07,6.11969e+06,0.00111101,0.00114355,0.0018537,None,0.0106162,0.011432,0.012379
34,model_cnn_internal_grid_False_7_5_60_3_200501_...,model_cnn_internal_grid_False_7_5_60_3_200501_13,h5,1.619864e+09,01.05.2021 13:12:41,20210501,model_cnn_internal_grid_False_7_5_60_3_200501_...,3,"[5, 10, 15]",7,...,9.72935e+06,1.54038e+07,7.26686e+06,0.00101182,0.00104661,0.00168676,None,0.00916022,0.012978,0.012255
65,model_spider6_gridTrue_0.0001_3_10_64.h5,model_spider6_gridTrue_0.0001_3_10_64,h5,1.619094e+09,22.04.2021 15:18:54,20210422,model_spider6_gridTrue_0.0001_3_10_64.csv,6,"[10, 20, 40, 50, 60, 70]",3,...,1.0876e+07,1.62831e+07,7.42926e+06,0.00122939,0.00121516,0.00177281,None,0.0057955,0.007069,0.010667
87,model_spider6_gridTrue_1e-05_5_5_64.h5,model_spider6_gridTrue_1e-05_5_5_64,h5,1.618417e+09,14.04.2021 19:14:58,20210414,model_spider6_gridTrue_1e-05_5_5_64.csv,6,"[5, 10, 20, 25, 30, 35]",5,...,1.06743e+07,1.63737e+07,6.36764e+06,0.0011502,0.00127321,0.00167457,None,0.0100169,0.009535,0.012087
0,model_dense_PrivatData_c4c4_k5_t0.5_w3.h5,model_dense_PrivatData_c4c4_k5_t0.5_w3,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_dense_PrivatData_c4c4_k5_t0.5_w3.csv,None,None,None,...,None,None,None,None,None,None,None,None,NaN,NaN
1,model_dense_PrivateData_c3c3_k10_t0.5_w5.h5,model_dense_PrivateData_c3c3_k10_t0.5_w5,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_dense_PrivateData_c3c3_k10_t0.5_w5.csv,None,None,None,...,None,None,None,None,None,None,None,None,NaN,NaN
2,model_fDenseNet6_PrivateData_with_B_tests_0.00...,model_fDenseNet6_PrivateData_with_B_tests_0.00...,h5,1.620732e+09,11.05.2021 14:14:51,20210511,model_fDenseNet6_PrivateData_with_B_tests_0.00...,None,None,None,...,None,None,None,None,None,None,None,None,NaN,NaN


In [ ]:
dd_all.head()

In [265]:
dd_all.to_excel('BEST_APS_Privat_model_20210511.xlsx')

In [140]:
model_X.get_config()['layers'][0]['class_name']

'InputLayer'

In [ ]:
model_cnn.save('model_cnn_8_0924_51.h5')

In [179]:
model_X.optimizer.get_config()['learning_rate']

0.002

In [130]:
history_XX = pd.read_csv(dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0])

In [129]:
dd_all[dd_all['full_name'] == mod]['csv_name'].to_list()[0]

'model_cnn_3_0920_2dense.csv'

In [132]:
history_XX.head()

,Unnamed: 0,loss,accuracy,auc,precision,recall,val_loss,val_accuracy,val_auc,val_precision,val_recall
0,0,0.006134,0.988309,0.998506,0.988309,0.988309,0.056956,0.986467,0.997934,0.986467,0.986467
1,1,0.006166,0.987957,0.998406,0.987957,0.987957,0.058530,0.985955,0.997500,0.985955,0.985955
2,2,0.006104,0.988153,0.998491,0.988153,0.988153,0.046868,0.988080,0.998440,0.988080,0.988080
3,3,0.006150,0.988304,0.998512,0.988304,0.988304,0.045066,0.988657,0.998293,0.988657,0.988657
4,4,0.006123,0.988003,0.998463,0.988003,0.988003,0.066977,0.982147,0.997378,0.982147,0.982147
